In [4]:
import numpy as np
import pandas as pd 
import yfinance as yf

In [5]:
data = yf.download('AAPL', start='2020-01-01', end='2023-01-01')

[*********************100%***********************]  1 of 1 completed


In [6]:
data['Daily Return'] = data['Adj Close'].pct_change()

In [7]:
avg_daily_return = data['Daily Return'].mean()
std_daily_return = data['Daily Return'].std()

In [8]:
risk_free_rate = 0.05

In [9]:
sharpe_ratio = (avg_daily_return - risk_free_rate / 252) / std_daily_return
annual_sharpe_ratio = sharpe_ratio * np.sqrt(252)
print(f'Annual Sharpe Ratio: {annual_sharpe_ratio}')

Annual Sharpe Ratio: 0.5632049743217762


CROSS SECTIONAL MOMENTUM


In [10]:
# Tải dữ liệu của nhiều cổ phiếu (có thể thay đổi danh sách này)
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NFLX', 'META']

data = yf.download(tickers, start='2020-01-01', end='2024-01-01')['Adj Close']

# Tính toán lợi suất hàng ngày (daily returns)
returns = data.pct_change().dropna()

# Tính hiệu suất của mỗi cổ phiếu trong 12 tháng trước đó (252 ngày giao dịch)
lookback_period = 252
momentum_returns = returns.rolling(window=lookback_period).apply(np.sum)

# Lấy dữ liệu hiệu suất của tháng gần nhất để xếp hạng
momentum_last_month = momentum_returns.iloc[-1]

[*********************100%***********************]  7 of 7 completed


In [11]:
# Xếp hạng các cổ phiếu dựa trên lợi suất trong 12 tháng trước đó
top_n = 3  # Số cổ phiếu có hiệu suất tốt nhất
bottom_n = 3  # Số cổ phiếu có hiệu suất kém nhất

# Sắp xếp các cổ phiếu dựa trên hiệu suất
ranked_stocks = momentum_last_month.sort_values(ascending=False)

# Lựa chọn cổ phiếu tốt nhất và kém nhất
winners = ranked_stocks.head(top_n).index  # Mua các cổ phiếu tốt nhất
losers = ranked_stocks.tail(bottom_n).index  # Bán các cổ phiếu kém nhất

print(f"Winners (Buy): {winners}")
print(f"Losers (Sell): {losers}")


Winners (Buy): Index(['META', 'TSLA', 'AMZN'], dtype='object', name='Ticker')
Losers (Sell): Index(['GOOGL', 'MSFT', 'AAPL'], dtype='object', name='Ticker')


In [12]:
# Tính lợi suất của nhóm cổ phiếu thắng (winners) và thua (losers) trong tháng tiếp theo
future_returns = returns.iloc[-21:]  # Lợi suất trong tháng tiếp theo (21 ngày giao dịch)

# Lợi nhuận trung bình của các cổ phiếu mua và bán
winners_return = future_returns[winners].mean(axis=1)
losers_return = future_returns[losers].mean(axis=1)

# Lợi nhuận của chiến lược cross-sectional momentum
momentum_strategy_return = winners_return - losers_return

# Tổng lợi suất chiến lược
cumulative_return = (1 + momentum_strategy_return).cumprod()

print(f"Lợi suất chiến lược cumulative return:\n{cumulative_return}")


Lợi suất chiến lược cumulative return:
Date
2023-11-30 00:00:00+00:00    0.993866
2023-12-01 00:00:00+00:00    0.995186
2023-12-04 00:00:00+00:00    0.995240
2023-12-05 00:00:00+00:00    0.988102
2023-12-06 00:00:00+00:00    0.990439
2023-12-07 00:00:00+00:00    0.987027
2023-12-08 00:00:00+00:00    0.995383
2023-12-11 00:00:00+00:00    0.989978
2023-12-12 00:00:00+00:00    0.995441
2023-12-13 00:00:00+00:00    0.996555
2023-12-14 00:00:00+00:00    1.016969
2023-12-15 00:00:00+00:00    1.022703
2023-12-18 00:00:00+00:00    1.032893
2023-12-19 00:00:00+00:00    1.040458
2023-12-20 00:00:00+00:00    1.023911
2023-12-21 00:00:00+00:00    1.035077
2023-12-22 00:00:00+00:00    1.029120
2023-12-26 00:00:00+00:00    1.036852
2023-12-27 00:00:00+00:00    1.049296
2023-12-28 00:00:00+00:00    1.037254
2023-12-29 00:00:00+00:00    1.025889
dtype: float64


In [16]:
# Giả sử lãi suất phi rủi ro là 0.02/năm (tính ra hàng ngày là 0.02/252)
risk_free_rate = 0.02/ (3 * 365)

# Tính Sharpe Ratio
excess_returns = returns - risk_free_rate  # Lợi suất vượt mức
mean_excess_return = excess_returns.mean()  # Lợi suất vượt mức trung bình
std_excess_return = excess_returns.std()  # Độ lệch chuẩn của lợi suất vượt mức

sharpe_ratio = mean_excess_return / std_excess_return
print(f"Sharpe Ratio: {sharpe_ratio}")

Sharpe Ratio: Ticker
AAPL     0.055250
AMZN     0.030814
GOOGL    0.043328
META     0.032057
MSFT     0.052388
NFLX     0.027990
TSLA     0.071127
dtype: float64


In [4]:
from datetime import datetime
import pandas as pd
import requests
import time

def to_timestamp(date_string):
    return int(datetime.strptime(date_string, "%d-%m-%Y %H:%M").timestamp())

def to_utc_date(timestamp):
    return datetime.utcfromtimestamp(int(timestamp)).strftime('%d-%m-%Y %H:%M')

def current_timestamp():
    return int(datetime.now().timestamp())

def progressBar(iterable, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    total = len(iterable)
    def printProgressBar (iteration):
        percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
        filledLength = int(length * iteration // total)
        bar = fill * filledLength + '-' * (length - filledLength)
        print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    printProgressBar(0)
    for i, item in enumerate(iterable):
        yield item
        printProgressBar(i + 1)
    print()

if __name__ == '__main__':
    start = '01-09-2020 00:00'
    end = None

    currency_pair = 'BTC'
    pair_sufix = 'inverse'
    interval = 1
    wait_count = 3

    start_date = to_timestamp(start)
    end_date = current_timestamp() if end == None else to_timestamp(end)
    if end == None:
        end = to_utc_date(end_date)
    filename = f'BYBIT-{currency_pair}-{pair_sufix}-{interval}m-data-from-{start.replace(" ", "_")}-to-{end.replace(" ", "_")}.csv'

    print(f'Downloading dataset {currency_pair} {pair_sufix} in candles of {interval}m from {start} to {end}')
    print("Total number of candles is",int((int(end_date)-int(start_date))/ (60 * interval)))

    list_candle_times = list(range(start_date, end_date, int(200 * 60 * interval)))
    data = []

    for current_time in progressBar(list_candle_times, prefix = 'Progress:', suffix = 'Complete', length = 50):
        status_code = 0
        response = 1

        while(status_code != 200):
            response = requests.get(f'https://api.bybit.com/v2/public/kline/list?symbol={currency_pair}&interval={interval}&limit=200&from={current_time}')
            status_code = response.status_code
            time.sleep(0.1)

        if response.json()["result"]:
            first_candle = response.json()["result"][0]
            if end_date <= first_candle["open_time"]:
                break
            data.append([first_candle["open_time"], first_candle["open"], first_candle["high"], first_candle["low"], first_candle["close"], first_candle["volume"], int(first_candle["open_time"]) + interval * 60])
            break  # Thoát khỏi vòng lặp sau khi lấy được chỉ số đầu tiên

    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time'])

    # Saving the CSV
    df.set_index('timestamp', inplace=True)
    df.to_csv(filename, sep=",", encoding='utf-8', index='timestamp')

    print("Download finished !!!")


C:\Users\tranm\AppData\Local\Temp\ipykernel_25460\3968608555.py:10: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.utcfromtimestamp(int(timestamp)).strftime('%d-%m-%Y %H:%M')


Total number of candles is 2171405


KeyboardInterrupt: 

In [6]:
! pip install pybit


  Using cached pybit-5.8.0-py2.py3-none-any.whl
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached pycryptodome-3.21.0-cp36-abi3-win_amd64.whl.metadata (3.4 kB)
Using cached pycryptodome-3.21.0-cp36-abi3-win_amd64.whl (1.8 MB)
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)


In [12]:
! pip install requests

In [7]:
from pybit.unified_trading import HTTP

In [ ]:
session = HTTP(
    testnet=False,
    api_key="epVMQ9vqe4IXIBX5CL",
    api_secret="5GSEdwSOR86mSeTs2EE0MH4dKCaObBNHXObC",
)

In [14]:
import requests

# Initialize the session
session = requests.Session()

# Define the API endpoint for the order book
url = "https://api.binance.com/api/v3/depth"
params = {
    "symbol": "BTCUSDT",
    "limit": 100  # Number of orders to retrieve
}

# Get the order book
response = session.get(url, params=params)
orderbook = response.json()
print(orderbook)

# Create five long USDC Options orders
payload = {"category": "option"}
orders = [{
    "symbol": "BTC-30JUN23-20000-C",
    "side": "Buy",
    "orderType": "Limit",
    "qty": "0.1",
    "price": i,
} for i in [15000, 15500, 16000, 16500, 16600]]

payload["request"] = orders

# Submit the orders in bulk (assuming you have a method to do this)
# response = session.place_batch_order(payload)
# print(response)


{'lastUpdateId': 53080884922, 'bids': [['67077.26000000', '2.83088000'], ['67077.14000000', '0.61571000'], ['67077.01000000', '0.00008000'], ['67077.00000000', '0.00099000'], ['67076.03000000', '0.00008000'], ['67076.02000000', '0.01475000'], ['67076.01000000', '0.16625000'], ['67076.00000000', '0.12580000'], ['67075.47000000', '0.00638000'], ['67075.36000000', '0.00008000'], ['67075.00000000', '0.02982000'], ['67074.94000000', '0.23229000'], ['67074.00000000', '0.12580000'], ['67073.40000000', '0.03560000'], ['67073.24000000', '0.00037000'], ['67072.45000000', '0.07914000'], ['67072.32000000', '0.00016000'], ['67072.01000000', '0.13778000'], ['67072.00000000', '0.12580000'], ['67071.90000000', '0.19754000'], ['67071.72000000', '0.00149000'], ['67071.52000000', '0.00100000'], ['67070.62000000', '0.00666000'], ['67070.00000000', '0.02934000'], ['67069.27000000', '0.01000000'], ['67068.96000000', '0.54866000'], ['67068.89000000', '0.32915000'], ['67068.69000000', '0.00012000'], ['67068.6

Order book for BTC-01DEC23:
{'code': -1121, 'msg': 'Invalid symbol.'}


Order book for BTC-01MAR24:
{'code': -1121, 'msg': 'Invalid symbol.'}


Order book for BTC-01NOV24:
{'code': -1121, 'msg': 'Invalid symbol.'}


Order book for BTC-01SEP23:
{'code': -1121, 'msg': 'Invalid symbol.'}


Order book for BTC-02AUG24:
{'code': -1121, 'msg': 'Invalid symbol.'}


Order book for BTC-02FEB24:
{'code': -1121, 'msg': 'Invalid symbol.'}


Order book for BTC-02JUN23:
{'code': -1121, 'msg': 'Invalid symbol.'}


Order book for BTC-03MAY24:
{'code': -1121, 'msg': 'Invalid symbol.'}


Order book for BTC-03NOV23:
{'code': -1121, 'msg': 'Invalid symbol.'}


Order book for BTC-04AUG23:
{'code': -1121, 'msg': 'Invalid symbol.'}


Order book for BTC-04OCT24:
{'code': -1121, 'msg': 'Invalid symbol.'}


Order book for BTC-05APR24:
{'code': -1121, 'msg': 'Invalid symbol.'}


Order book for BTC-05JAN24:
{'code': -1121, 'msg': 'Invalid symbol.'}


Order book for BTC-05JUL24:
{'code': -1121, 'msg': 'Invalid symb

In [14]:
# Ryuryu's Bybit Historical Data Downloader 
# (Production Mode 6973)
# -------------------------------------
# (c) 2022 Ryan Hayabusa 
# Github: https://github.com/ryu878 
# Web: https://aadresearch.xyz/
# Discord: ryuryu#4087
# -------------------------------------
# pip install beautifulsoup4
# pip install requests

import urllib.request
import os
import re
import gzip
import time
import requests
from bs4 import BeautifulSoup


# Set the file version
ver = '1.3:02/05/23'

# Define the base URL
base_url = 'https://public.bybit.com/trading/'

# Select the start date
start_date = '2024-10-10'

# Set the list of coins
coins = []

# Create a function to download the files
def download_file(url, local_path):
    with urllib.request.urlopen(url) as response, open(local_path, 'wb') as out_file:
        data = response.read()
        out_file.write(data)


# Create a function to check if a file exists
def file_exists(local_path):
    return os.path.exists(local_path)


# Make a GET request to the base URL and parse the HTML
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all the links on the page
texts = ['BTCPERP/']
links = soup.find_all('a', string=lambda text: text in texts)
# 'BTCUSD/', 'BTCUSDT/', 

# Loop through all the links
for link in links:
    # Get the href attribute of the link
    href = link.get('href')
    # Check if the href attribute is a directory
    if href.endswith('/'):
        # Get the directory name
        dir_name = href[:-1]
        # Create the directory locally if it doesn't exist
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        # Make a GET request to the directory URL and parse the HTML
        dir_url = base_url + href
        dir_response = requests.get(dir_url)
        dir_soup = BeautifulSoup(dir_response.text, 'html.parser')
        # Find all the CSV files in the directory
        csv_links = dir_soup.find_all(href=re.compile('.csv.gz$'))
        # Loop through all the CSV files
        for csv_link in csv_links:
            # Get the CSV file name
            csv_name = csv_link.text
            # Extract the date from the CSV file name
            csv_date = re.findall(r'\d{4}-\d{2}-\d{2}', csv_name)[0]
            # Check if the file is from or after the selected start date
            if csv_date >= start_date:
                # Construct the full URL of the CSV file
                csv_url = dir_url + csv_name
                # Construct the local path of the extracted file
                extracted_path = os.path.join(dir_name, csv_name[:-3])
                # Check if the extracted file exists locally
                if file_exists(extracted_path):
                    print('Skipping download of', csv_name, '- extracted file already exists.')
                else:
                    # Construct the local path of the archive file
                    archive_path = os.path.join(dir_name, csv_name)
                    # Download the archive file if it doesn't exist locally
                    if not file_exists(archive_path):
                        download_file(csv_url, archive_path)
                        print('Downloaded:', archive_path)
                        time.sleep(0.1)
                    # Check if the file is a gzip archive
                    if csv_name.endswith('.gz'):
                        # Open the gzip archive and extract the contents
                        with gzip.open(archive_path, 'rb') as f_in:
                            with open(extracted_path, 'wb') as f_out:
                                f_out.write(f_in.read())
                                print('Extracted:', extracted_path)
                        # Remove the archive file
                        os.remove(archive_path)
                        print('Removed:', archive_path)
                    else:
                        # Rename the file to remove the .csv extension
                        os.rename(archive_path, extracted_path)
                        print('Renamed:', archive_path, 'to', extracted_path)
            else:
                # Skip the file
                print('Skipping download of', csv_name, '- date is before start date.')

Skipping download of BTCPERP2023-05-29.csv.gz - date is before start date.
Skipping download of BTCPERP2023-05-30.csv.gz - date is before start date.
Skipping download of BTCPERP2023-05-31.csv.gz - date is before start date.
Skipping download of BTCPERP2023-06-01.csv.gz - date is before start date.
Skipping download of BTCPERP2023-06-02.csv.gz - date is before start date.
Skipping download of BTCPERP2023-06-03.csv.gz - date is before start date.
Skipping download of BTCPERP2023-06-04.csv.gz - date is before start date.
Skipping download of BTCPERP2023-06-05.csv.gz - date is before start date.
Skipping download of BTCPERP2023-06-06.csv.gz - date is before start date.
Skipping download of BTCPERP2023-06-07.csv.gz - date is before start date.
Skipping download of BTCPERP2023-06-08.csv.gz - date is before start date.
Skipping download of BTCPERP2023-06-09.csv.gz - date is before start date.
Skipping download of BTCPERP2023-06-10.csv.gz - date is before start date.
Skipping download of BTCP

In [5]:
import urllib.request
import os
import re
import gzip
import time
import requests
from bs4 import BeautifulSoup


# Set the file version
ver = '1.3:02/05/23'

# Define the base URL
base_url = 'https://public.bybit.com/kline_for_metatrader4/'

# Select the start date
start_date = '2024-6-1'

# Set the list of coins
coins = []

# Create a function to download the files
def download_file(url, local_path):
    with urllib.request.urlopen(url) as response, open(local_path, 'wb') as out_file:
        data = response.read()
        out_file.write(data)


# Create a function to check if a file exists
def file_exists(local_path):
    return os.path.exists(local_path)


# Make a GET request to the base URL and parse the HTML
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all the links on the page
texts = ['BTCUSDT/']
links = soup.find_all('a', string=lambda text: text in texts)
# 'BTCUSD/', 'BTCUSDT/', 

# Loop through all the links
for link in links:
    # Get the href attribute of the link
    href = link.get('href')
    # Check if the href attribute is a directory
    if href.endswith('/'):
        # Get the directory name
        dir_name = href[:-1]
        # Create the directory locally if it doesn't exist
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        # Make a GET request to the directory URL and parse the HTML
        dir_url = base_url + href 
        dir_response = requests.get(dir_url)
        dir_soup = BeautifulSoup(dir_response.text, 'html.parser')
        # Find all the CSV files in the directory
        csv_links = dir_soup.find_all(href=re.compile('.csv.gz$'))
        # Loop through all the CSV files
        for csv_link in csv_links:
            # Get the CSV file name
            csv_name = csv_link.text
            # Extract the date from the CSV file name
            csv_date = re.findall(r'\d{4}-\d{2}-\d{2}', csv_name)[0]
            # Check if the file is from or after the selected start date
            if csv_date >= start_date:
                # Construct the full URL of the CSV file
                csv_url = dir_url + csv_name
                # Construct the local path of the extracted file
                extracted_path = os.path.join(dir_name, csv_name[:-3])
                # Check if the extracted file exists locally
                if file_exists(extracted_path):
                    print('Skipping download of', csv_name, '- extracted file already exists.')
                else:
                    # Construct the local path of the archive file
                    archive_path = os.path.join(dir_name, csv_name)
                    # Download the archive file if it doesn't exist locally
                    if not file_exists(archive_path):
                        download_file(csv_url, archive_path)
                        print('Downloaded:', archive_path)
                        time.sleep(0.1)
                    # Check if the file is a gzip archive
                    if csv_name.endswith('.gz'):
                        # Open the gzip archive and extract the contents
                        with gzip.open(archive_path, 'rb') as f_in:
                            with open(extracted_path, 'wb') as f_out:
                                f_out.write(f_in.read())
                                print('Extracted:', extracted_path)
                        # Remove the archive file
                        os.remove(archive_path)
                        print('Removed:', archive_path)
                    else:
                        # Rename the file to remove the .csv extension
                        os.rename(archive_path, extracted_path)
                        print('Renamed:', archive_path, 'to', extracted_path)
            else:
                # Skip the file
                print('Skipping download of', csv_name, '- date is before start date.')